In [1]:
import torch
import torch.nn.functional as F
from torchvision.datasets import MNIST

dataset = MNIST(root = 'data/', download = True)

100%|██████████| 9.91M/9.91M [01:02<00:00, 158kB/s] 
100%|██████████| 28.9k/28.9k [00:00<00:00, 172kB/s]
100%|██████████| 1.65M/1.65M [00:26<00:00, 62.3kB/s]
100%|██████████| 4.54k/4.54k [00:00<?, ?B/s]
